# Import

In [1]:
# import all packages
import pandas as pd
import numpy as np
import psycopg2 as psycopg2
import sql_functions as sqlf
import string as string

In [16]:
# pd.set_option("display.max.rows", 150)

In [3]:
schema = "capstone_24_4_group1"

In [4]:
countries_query = f'''   SELECT *
                    FROM {schema}."countries"
                    '''
bx_infl_query = f'''   SELECT *
                    FROM {schema}."box_office_data_inflated"
                    '''

In [5]:
countries_df = sqlf.get_dataframe(countries_query)
bx_infl_df = sqlf.get_dataframe(bx_infl_query)

In [ ]:
display(countries_df.head(5))
display(bx_infl_df.head(5))

In [ ]:
display(pd.DataFrame(countries_df["long_name"].unique()))

In [ ]:
display(pd.DataFrame(bx_infl_df["country"].unique()))

In [9]:
bx_csv = pd.read_csv("Data/bx_countries.csv", sep=";")

In [ ]:
bx_csv["long_name"].unique()

In [11]:
bx_infl_new = bx_infl_df.merge(bx_csv, how= "left", left_on="country", right_on="bx_country")

In [12]:
bx_infl_new.drop(columns=['country', 'bx_country'], inplace=True)

In [13]:
bx_infl_new = bx_infl_new[["tconst","long_name", "values", "inflated_values"]]

In [14]:
bx_infl_new = bx_infl_new.rename(columns={'long_name': 'country'})

In [ ]:
pd.DataFrame(bx_infl_new["country"].unique())

# Upload

In [18]:
schema

'capstone_24_4_group1'

In [19]:
dataframes = {
    "box_office_data_inflated":bx_infl_new,
}

In [20]:
engine = sqlf.get_engine()
engine

# Write records stored in a dataframe to SQL database
if engine != None:
    try:
        for table_name, df in dataframes.items():
            df.to_sql(table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='replace', # Drop the table before inserting new values 
                            schema=schema, # your class schema
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print("nope") 

The box_office_data_inflated table was imported successfully.
